In [73]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "b8d6d964989341a2bfe0893b4c769ecd"
endpoint = "https://20221127-yy.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

## 文字の認識

In [74]:
# Get an image with text
read_image_url = "https://raw.githubusercontent.com/MicrosoftDocs/azure-docs/master/articles/cognitive-services/Computer-vision/Images/readsample.jpg"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - remote
'''

print("End of Computer Vision quickstart.")

The quick brown fox jumps
[87.0, 653.0, 2574.0, 712.0, 2571.0, 865.0, 83.0, 820.0]
over the lazy dog!
[177.0, 1008.0, 1988.0, 1010.0, 1987.0, 1159.0, 177.0, 1150.0]

End of Computer Vision quickstart.


In [75]:
description_results = computervision_client.describe_image(read_image_url)
description_results

In [76]:
for caption in description_results.captions:
    print(caption)

{'additional_properties': {}, 'text': 'diagram', 'confidence': 0.7789058685302734}


## 画像の描写

In [77]:
# images_folder = os.path.join (os.path.dirname(os.path.abspath(__file__)), "images")
remote_image_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg"
'''
END - Quickstart variables
'''


'''
Tag an Image - remote
This example returns a tag (key word) for each thing in the image.
'''
print("===== Tag an image - remote =====")
# Call API with remote image
tags_result_remote = computervision_client.tag_image(remote_image_url )

# Print results with confidence score
print("Tags in the remote image: ")
if (len(tags_result_remote.tags) == 0):
    print("No tags detected.")
else:
    for tag in tags_result_remote.tags:
        print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))
print()
'''
END - Tag an Image - remote
'''
print("End of Computer Vision quickstart.")

===== Tag an image - remote =====
Tags in the remote image: 
'outdoor' with confidence 99.00%
'building' with confidence 98.81%
'sky' with confidence 98.21%
'stadium' with confidence 98.17%
'ancient rome' with confidence 96.16%
'ruins' with confidence 95.04%
'amphitheatre' with confidence 93.99%
'ancient roman architecture' with confidence 92.65%
'historic site' with confidence 89.55%
'ancient history' with confidence 89.54%
'history' with confidence 86.72%
'archaeological site' with confidence 84.41%
'travel' with confidence 65.85%
'large' with confidence 61.02%
'city' with confidence 56.57%

End of Computer Vision quickstart.


## 画像カテゴリの取得

In [78]:
remote_image_features = ["categories"]
categorize_results_remote = computervision_client.analyze_image(remote_image_url, remote_image_features)

In [79]:
for category in categorize_results_remote.categories:
    print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))

'building_' with confidence 31.64%
'others_' with confidence 0.39%
'outdoor_' with confidence 3.91%


## 画像タグの取得

In [80]:
tags_results_remote = computervision_client.tag_image(remote_image_url)
for tag in tags_results_remote.tags:
    print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

'outdoor' with confidence 99.00%
'building' with confidence 98.81%
'sky' with confidence 98.21%
'stadium' with confidence 98.17%
'ancient rome' with confidence 96.16%
'ruins' with confidence 95.04%
'amphitheatre' with confidence 93.99%
'ancient roman architecture' with confidence 92.65%
'historic site' with confidence 89.55%
'ancient history' with confidence 89.54%
'history' with confidence 86.72%
'archaeological site' with confidence 84.41%
'travel' with confidence 65.85%
'large' with confidence 61.02%
'city' with confidence 56.57%


## 物体を検出

In [81]:
remote_image_url_objects = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg"
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url_objects)

In [90]:
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
        print("object at location {}, {}, {}, {}".format( \
                object.rectangle.x, object.rectangle.x + object.rectangle.w, \
                object.rectangle.y, object.rectangle.y + object.rectangle.h))
        print(object.object_property)
        

object at location 213, 365, 85, 208
shirts
object at location 218, 402, 179, 384
Jeans
object at location 238, 417, 298, 416
Skateboard
object at location 116, 419, 60, 386
person


# ローカルファイルに対応させる

In [83]:
def detect_objects(filepath):
    local_image = open(filepath, "rb")
    detect_objects_results = computervision_client.detect_objects_in_stream(local_image)
    objects = detect_objects_results.objects
    return objects

In [84]:
def get_tags(filepath):
    local_image = open(filepath, "rb")
    tags_results = computervision_client.tag_image_in_stream(local_image)
    tags = tags_results.tags
    tags_name = []
    for tag in tags:
        tags_name.append(tag.name)
    return tags_name
    

In [85]:
filepath = "sample01.jpg"
get_tags(filepath)

['tableware',
 'food',
 'baked goods',
 'plate',
 'drink',
 'coffee cup',
 'dishware',
 'saucer',
 'snack',
 'serveware',
 'meal',
 'mug',
 'tea',
 'fast food',
 'breakfast',
 'fork',
 'kitchen utensil',
 'dish',
 'brunch',
 'platter',
 'dessert',
 'cup',
 'coffee',
 'indoor',
 'sitting',
 'table']

In [86]:
filepath = "sample01.jpg"
objects = detect_objects(filepath)

In [87]:
objects[0].rectangle.x

879